In [1]:
import pyvisa as visa
#import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from labview_automation import LabVIEW
from MultiPyVu import MultiVuServer as mvs
from MultiPyVu import MultiVuClient as mvc

        
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=20, skip_points=10, time_per_point=0.02):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v




import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise
        
def set_Field(field=0, persistent=True):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
def trigger_ac(ac, offs=0.005, amp=1e-6, duration=0.005, wait_after_arm=2):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(offs))
    ac.write("SOUR:WAVE:AMPL {}".format(amp))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(duration))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(wait_after_arm)
    ac.write("SOUR:WAVE:INIT")
    

    
def init_communication(amp=0.00005):
    print("check ac_1: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_1, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    
    print("check ac_2: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_2, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    """
    print("check ac_mid: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_mid, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)"""
    
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = amp            # Sets the source current to 0.1 mA
    dc.enable_source() 
    print("enable dc current now, current:" + str(dc))
    print("check nano volt:")
    for i in range(10):
        print(volt.query("fetch?"))
        time.sleep(0.1)
    time.sleep(10)
    dc.shutdown()   
    print("dc current disabled")

    
def do_measurement_three_pulses(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "data/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=ac_current, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""



def LVread():
    lv = LabVIEW()
    lv.start() # Launches the active LabVIEW with the listener VI
    with lv.client() as c:
        control_values = {
        
            }
    
        indicators = c.run_vi_synchronous(r"D:\User Data\Cheng Yang\Sam files\_switch\test_labview_python\\PPMS_Status_Eckberg.vi", control_values)
    #  print(indicators['Temp (K)'],indicators['Angle'], indicators['Field'] )
    #lv.kill() # Stop LabVIEW
    T=float(indicators['Temp (K)'])
    Ang=float(indicators['Angle'])
    Field=float(indicators['Field'])
    return T, Ang, Field



    
def do_measurement_one_pulse_FM(sample_id='', ac_current=[1*1e-3], dc_current=100e-6, duration=0.001,
                                wait_after_trigger=1, pulse=None):
    file_name = "data/{}ac_{}ms_wait{}s_{}.csv".format(sample_id, 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("current,ac_source,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{}\n".format(0, 'init', average_v, std_v, dc_current))

    for i in ac_current:
        print("pulse current=:{} A".format(i))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            if current == 'ac_1':
                    trigger_ac(ac_1, offs=i, duration=duration)
            elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * i, duration=duration)
            elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=i, duration=duration)
            elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * i, duration=duration)
            elif current == 'ac_2':
                    trigger_ac(ac_2, offs=i, duration=duration)
            elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * i, duration=duration)
            else:
                    print("no ac " + current)
                    raise
                    
            time.sleep(wait_after_trigger)
            average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
            with open(file_name, 'a') as f:
                f.write("{},{},{},{},{}\n".format(i, current, average_v, std_v, dc_current))
            print(average_v)
            time.sleep(0.1)
                
rm = visa.ResourceManager()
rm.list_resources()


('ASRL1::INSTR',
 'GPIB0::15::INSTR',
 'GPIB1::7::INSTR',
 'GPIB1::8::INSTR',
 'GPIB1::10::INSTR',
 'GPIB1::11::INSTR',
 'GPIB1::12::INSTR')

In [21]:
def SetField(Field=200, rate=100, Mode='persistent'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='persistent':
                client.set_field(Field,
                         rate, 
                         client.field.approach_mode.no_overshoot,
                         client.field.driven_mode.persistent)
            elif  Mode=='driven':
                client.set_field(Field,
                         rate, 
                         client.field.approach_mode.no_overshoot,
                         client.field.driven_mode.driven)
            else:
                print("wrong mode")
                
                
def GetField():
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
    
            Field, sF = client.get_field()
        
    return Field, sF 


def SetTemp(Temp=2, rate=10, Mode='fast_settle'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='fast_settle':
                client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.fast_settle)
            elif  Mode=='no_overshoot':
                 client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.no_overshoot)
            else:
                print("wrong mode")
def GetTemp():
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
    
            Temp, sT = client.get_temperature()
        
    return Temp, sT


def SetTempStable(Temp=2, rate=10, Mode='fast_settle'):
    with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='fast_settle':
                client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.fast_settle)
            elif  Mode=='no_overshoot':
                 client.set_temperature(Temp,
                         rate, 
                         client.temperature.approach_mode.no_overshoot)
            else:
                print("wrong mode")
            
            
            Flg = 1
            while  Flg:
                time.sleep(60)
                T = client.get_temperature()
                if abs(T[0] - Temp)<0.1:
                    Flg = 0
            
            time.sleep(60)
            
            Flg = 1
            while  Flg:
                time.sleep(60)
                T = client.get_temperature()
                if abs(T[0] - Temp)<0.1:
                    print('Temp is {}'.format(Temp))
                    Flg = 0
                    
def SetFieldStable(Field=200, rate=10, Mode='persistent'):
     with mvs.MultiVuServer() as server:

    # Start the client
        with mvc.MultiVuClient() as client:
            if Mode=='persistent':
                client.set_field(Field,
                             rate, 
                             client.field.approach_mode.no_overshoot,
                             client.field.driven_mode.persistent)
            elif Mode=='driven':
                client.set_field(Field,
                             rate, 
                             client.field.approach_mode.no_overshoot,
                             client.field.driven_mode.driven)
            else:
                print("wrong mode")
                  
            Flg = 1
            while Flg:
                time.sleep(1)
                F = client.get_field()
                if F[1]=='Holding (driven)' and abs(F[0]-Field)<1:
                    Flg = 0
                if F[1]=='Stable' and abs(F[0]-Field)<1:
                    Flg = 0   
            
            print("field is stable = {}Oe".format(F[0]))
            
            


In [3]:
sensitivity = {1:26, 0.5:25, 0.2:24, 0.1:23, 0.05:22, 0.02:21, 0.01:20, 0.005:19, 0.002:18, 0.001:17, 0.0005:16, 0.0002:15,
               0.0001:14, 0.00005:13, 0.00002:12, 0.00001:11, 0.000005:10, 0.000002:9, 0.000001:8, 0.0000005:7, 0.0000002:6, 0.0000001:5, 0.00000005:4, 0.00000002:3,
               0.00000001:2, 0.000000005:1, 0.000000002:0}


class Lockin:
    def __init__(self, resource, gpib=1):
        self.lock_in = resource.open_resource('GPIB1::{}::INSTR'.format(gpib))

    def change_harmonic(self, harm=1, sens=1):

       
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SENS {}".format(sensitivity[sens]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 1").strip())
        y = float(self.lock_in.query("OUTP? 2").strip())
        r = float(self.lock_in.query("OUTP? 3").strip())
        theta = float(self.lock_in.query("OUTP? 4").strip())
        return x, y, r, theta

    def lock_in_measure(self, count=10, time_step=0.1,
                        wait_before_measure=1,
                        harm=1, sens=0.00001):
#        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)
            
        xs = sorted(xs)
        ys=  sorted(ys)
        rs=  sorted(rs)
        thetas=sorted(thetas)
        
        if len(xs)>30:
            xs = xs[round(1/6*len(xs)): -round(1/6*len(xs))]
            ys = ys[round(1/6*len(ys)): -round(1/6*len(ys))]
            rs = rs[round(1/6*len(rs)): -round(1/6*len(rs))]
            thetas = thetas[round(1/6*len(thetas)): -round(1/6*len(thetas))]

        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)
        
        
        return x_mean, y_mean, r_mean, theta_mean, \
               x_std, y_std,r_std, theta_std

In [4]:
#switch
ac_6221_1_id = 12
dc_2400_id = 24


dc = Keithley2400("GPIB1::{}".format(dc_2400_id))


ac_1 = rm.open_resource('GPIB1::{}::INSTR'.format(ac_6221_1_id))

lock1 = Lockin(rm, 7)
lock2 = Lockin(rm, 8)
lock3 = Lockin(rm, 10)
lock4 = Lockin(rm, 11)

In [180]:
print(lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))
print(lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))
print(lock3.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))
print(lock4.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1))

(0.000864033, 9.38182e-05, 0.000869099, 6.198, 0.0, 0.0, 0.0, 0.0)
(-4.24336e-08, -5.25035e-08, 6.74049e-08, -128.88, 0.0, 0.0, 0.0, 0.0)
(-5.27016e-07, -1.78349e-07, 5.56003e-07, -161.382, 0.0, 0.0, 0.0, 0.0)
(5.30856e-08, 2.90574e-07, 2.84987e-07, 95.232, 0.0, 0.0, 0.0, 0.0)


In [250]:
A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.1)
B=lock2.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0.1)
C=lock3.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0.1)
print(A[0])
print(B[1])
print(C[0])

0.000861172
-9.313749999999973e-11
-4.7361500000000005e-07


In [211]:
Field=np.append(np.arange(-90000,90000,1000),np.arange(90000,-91000,-1000)) 
Field=np.append(Field, 0) 
print(Field)

[-90000 -89000 -88000 -87000 -86000 -85000 -84000 -83000 -82000 -81000
 -80000 -79000 -78000 -77000 -76000 -75000 -74000 -73000 -72000 -71000
 -70000 -69000 -68000 -67000 -66000 -65000 -64000 -63000 -62000 -61000
 -60000 -59000 -58000 -57000 -56000 -55000 -54000 -53000 -52000 -51000
 -50000 -49000 -48000 -47000 -46000 -45000 -44000 -43000 -42000 -41000
 -40000 -39000 -38000 -37000 -36000 -35000 -34000 -33000 -32000 -31000
 -30000 -29000 -28000 -27000 -26000 -25000 -24000 -23000 -22000 -21000
 -20000 -19000 -18000 -17000 -16000 -15000 -14000 -13000 -12000 -11000
 -10000  -9000  -8000  -7000  -6000  -5000  -4000  -3000  -2000  -1000
      0   1000   2000   3000   4000   5000   6000   7000   8000   9000
  10000  11000  12000  13000  14000  15000  16000  17000  18000  19000
  20000  21000  22000  23000  24000  25000  26000  27000  28000  29000
  30000  31000  32000  33000  34000  35000  36000  37000  38000  39000
  40000  41000  42000  43000  44000  45000  46000  47000  48000  49000
  5000

In [22]:
Field=np.append(np.arange(-5000,5000,50),np.arange(5000,-5050,-50)) 
Field=np.append(Field, 0) 
# T=GetTemp()
# print('Temp is {} K'.format(T[0]))
T=300
curr=4
file_name = "data/LFO110_OOP_curr_para_Caxis_harm123_fielddep_{}K_{}.csv".format(T, int(time.time()))
with open(file_name, 'w') as f:
    f.write("Field,x1,y1,r1,phi1,dx1,dy1,dr1,dphi1,x2,y2,r2,phi2,dx2,dy2,dr2,dphi2,x3,y3,r3,phi3,dx3,dy3,dr3,dphi3,x4,y4,r4,phi4,dx4,dy4,dr4,dphi4,I\n")

for field in Field:
    SetFieldStable(Field=field, rate=100, Mode='driven')
    A=lock1.lock_in_measure(count=3, time_step=0.1, wait_before_measure=0)
    B=lock2.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0)
    C=lock3.lock_in_measure(count=10, time_step=0.1, wait_before_measure=0)
    D=lock4.lock_in_measure(count=1, time_step=0, wait_before_measure=0)
    
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(field, A[0], A[1], A[2], A[3], A[4], A[5], A[6], A[7], B[0], B[1], B[2], B[3], B[4], B[5], B[6], B[7], C[0], C[1], C[2], C[3], C[4], C[5], C[6], C[7], D[0], D[1], D[2], D[3], D[4], D[5], D[6], D[7], curr))


MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 52131)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 52131)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 52131)
MultiVuServer - Client ('127.0.0.1', 52131) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 52131)
MultiVuClient - Closing connection to ('127.0.0.1', 52131)


field is stable = -89999.6094Oe


MultiVuServer - Closing connection to ('127.0.0.1', 52131)
MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 52133)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 52133)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 52133)
MultiVuServer - Client ('127.0.0.1', 52133) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 52133)
MultiVuClient - Closing connection to ('127.0.0.1', 52133)


field is stable = -89000.6562Oe


MultiVuServer - Closing connection to ('127.0.0.1', 52133)
MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 52134)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 52134)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 52134)
MultiVuServer - Client ('127.0.0.1', 52134) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 52134)
MultiVuClient - Closing connection to ('127.0.0.1', 52134)


...

field is stable = 89000.375Oe


MultiVuServer - Closing connection to ('127.0.0.1', 55635)
MultiVuServer - Found PPMS running.
MultiVuServer - listening on ('localhost', 5000)
MultiVuServer - Press ESC to exit.
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuClient - Starting connection to ('localhost', 5000)
MultiVuServer - Accepted connection from ('127.0.0.1', 55637)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 55637)
MultiVuClient - Connected to PPMS MultiVuServer at ('127.0.0.1', 55637)
MultiVuServer - Client ('127.0.0.1', 55637) disconnected.
MultiVuClient - Closing connection to ('127.0.0.1', 55637)
MultiVuClient - Closing connection to ('127.0.0.1', 55637)


field is stable = 88000.2969Oe


MultiVuServer - Closing connection to ('127.0.0.1', 55637)


PermissionError: [Errno 13] Permission denied: 'data/LFO110_IP_curr_para_Caxis_harm123_fielddep_300K_1666931946.csv'